In [1]:
from transformers import pipeline
model_name="paust/pko-t5-base"

def generate_text(pipe, text, num_return_sequences=5, max_length=200):
  text = f"토픽, 감성 태깅 :{text}"
  out = pipe(text, num_return_sequences=num_return_sequences, max_length=max_length)
  return [x['generated_text'] for x in out]

def run_pipeline(model_path, text):
    nlg_pipeline = pipeline('text2text-generation', model=model_path, tokenizer=model_name)
    return (generate_text(nlg_pipeline, text, num_return_sequences=1, max_length=200)[0])

c:\Users\KHU\anaconda3\envs\tagging\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pandas as pd
test=pd.read_csv(r"C:\Users\KHU\Desktop\tagging_seq2seq\test_without_score.csv")

In [3]:
for row in test['sentence'].iloc[:100]:
    print("원본:", row)
    print(run_pipeline('without_score_model.pth', row))
    print()

원본: 기종: 아이폰12프로도킹이나 기존 선으로 연결하는 보조베터리보다 훨씬 이동성,편의성이 좋습니다
훨씬 이동성,편의성이 좋습니다 : (휴대성)(긍정)

원본: 맥세이프용 케이스인 경우엔 접착력도 우수하구요
맥세이프용 케이스인 경우엔 접착력도 우수하구요 : (맥세이프)(긍정)

원본: 다만, 3에서 충전을 시작해서 79까지가 최대충전인 점은 좀 의아하네요…;;
다만, 3에서 충전을 시작해서 79까지가 최대충전인 점은 좀 의아하네요 : (배터리용량)(부정)

원본: 거기다 충전시간도 한참걸려서(약 3-4시간) 고속이 아닌줄 알았는데 고속 충전기라고 명시되어 있네요
거기다 충전시간도 한참걸려서(약 3-4시간) 고속이 아닌줄 알았는데 고속 충전기라고 명시되어 있네요 : (고속충전)(부정)

원본: 제가 아무리 폰을 사용하면서 충전을 했더라도 기존 동일 용량의 보조베터리와 비교해보더러도 예상 이상으로 오래걸렸습니다
제가 아무리 폰을 사용하면서 충전을 했더라도 기존 동일 용량의 보조베터리와 비교해보더러도 예상 이상으로 오래걸렸습니다 : (배터리충전속도)(부정)

원본: 이것만 가지고 당일외출이 가능할지 고민이 되네요…
이것만 가지고 당일외출이 가능할지 고민이 되네요 : (기타)(부정)

원본: 원래 이런건지, 제가 받은 기기가 이상한지 잘 ㅁㅎ르겠네요ㅠ
원래 이런건지, 제가 받은 기기가 이상한지 잘 ᄆᄒ르겠네요 : (기타)(부정)

원본: 추가저도 스티커로 꾸며봤습니다 ㅎㅎ
저도 스티커로 꾸며봤습니다 ᄒᄒ : (디자인)(긍정)

원본: 흰색이 모서리부분쪽으로 이염이 좀 되더라구요
흰색이 모서리부분쪽으로 이염이 좀 되더라구요 : (색감)(부정)

원본: 그부분을 가릴 겸 방지할 겸해서 스티커를 붙여놨어요
그부분을 가릴 겸 방지할 겸해서 스티커를 붙여놨어요 : (기타)(부정)

원본: 경험상, 외출전에 폰에 적정수준 이상(약70-80)충전되어있었다면, 보조베터리로 당일 외출하기에 큰 문제는 없습니다
외출전에 폰에 적정수준 이상(약70-80)충전되어있었다면, 보조베터리로 

: 